In [13]:
import pandas as pd
import numpy as np

In [14]:
def is_shared(df1, df2):
    df1 = pd.read_csv(df1)
    df2 = pd.read_csv(df2)
    df1_top = df1.copy().head(10)
    df2_top = df2.copy().head(10)
    df1_top = df1_top["Principal Component"].to_list()
    df2_top = df2_top["Principal Component"].to_list()
    shared = [PC for PC in df1_top if PC in df2_top]
    shared_ = ""
    for par in shared:
        shared_+=par+", "
    print(shared_)

In [15]:
is_shared("Ranked_PCs_6.csv", "Ranked_PCs_9.csv")

PC-61, PC-5, PC-13, PC-3, PC-44, PC-34, 


In [18]:
def get_coeff(data, row):
    try:
        idx = list(np.where(data["Principal Component"] == row[0])[0])[0]
        value = data["Coefficient"][idx]
    except IndexError:
        value = "-"
        print(idx)
    finally:
        return value

def score_pc(row):
    score = 0
    num_cols = len(row)
    for n in range(1, num_cols):
        val = row[n]
        if val != "-":
            score+=val
    return score

def rank_scores(scores):
    from scipy import stats
    return len(scores)-stats.rankdata(scores, method='dense')
    
def rank_pc(row, scores, ranks):
    pc_score = row[-1]
    for score in scores:
        if score == pc_score:
            idx = scores.index(score)
            break
        else:   
            idx = None
    if idx is not None:
        rank = ranks[idx]+1
    else:
        rank = None
    return rank

def scale_scores(df, scores, pos=-1):
    df["Score"] = df.apply(
        lambda row: row[-1]/max(scores),
        axis=1
    )
    return df

def is_shared_all(dfs):
    all_top_pcs = []
    all_data = {}
    num_dfs = len(dfs)
    for df in dfs:
        data = pd.read_csv(df)
        data_top = data.copy().head(10)
        all_data[df] = data_top
        data_top = data_top["Principal Component"].to_list()
        all_top_pcs.extend(data_top)
    all_top_pcs_unique = []
    for pc in all_top_pcs:
        if pc not in all_top_pcs_unique:
            all_top_pcs_unique.append(pc)
    uniques_df = pd.DataFrame(
        all_top_pcs_unique,
        columns=["Principal Component"])
    for df in dfs:
        data = all_data[df]
        uniques_df["Model " + df.split(".")[0][-1]] = uniques_df.apply(
            lambda row: get_coeff(data, row),
            axis=1
        )
    uniques_df = uniques_df.drop("Model 6", axis=1)
    uniques_df["Score"] = uniques_df.apply(
        lambda row: score_pc(row),
        axis=1
    )
    """scores = uniques_df["Score"].values.tolist()
    uniques_df = scale_scores(uniques_df, scores)"""
    scores = uniques_df["Score"].values.tolist()
    ranks = rank_scores(scores)
    uniques_df["Rank"] = uniques_df.apply(
        lambda row: rank_pc(row, scores, ranks),
        axis=1
    )
    uniques_df.to_csv("outputs/all_top_shared_PCs_unordered.csv")
    uniques_df = uniques_df.sort_values(by="Rank", ascending=True)
    print(uniques_df)
    uniques_df.to_csv("outputs/all_top_shared_PCs.csv")
    print("All top unique PCs:\n" + str(all_top_pcs_unique)[1:-1])
    common_pcs = []
    for pc in all_top_pcs_unique:
        count = 0
        for pc_ in all_top_pcs:
            if pc == pc_:
                count+=1
        if count >= num_dfs/2:
            common_pcs.append(pc)
    print("Most common PCs:\n" + str(common_pcs))

    uniques_df_top = None
    df_count = 0
    for df in dfs:
        if df_count == 0:
            uniques_df_top = pd.read_csv(df)
        else:
            uniques_df_top = pd.merge(
                left=uniques_df_top,
                right=df,
                left_on="Principal Component",
                right_on="Principal Component"
            )
        count+=1
    pcs_ = uniques_df.head(10)["Principal Component"]
    uniques_df_top = pd.merge(
        left=uniques_df_top,
        right=pcs_,
        left_on="Principal Component",
        right_on="Principal Component"
    )
    uniques_df_top = uniques_df_top.drop("Coefficient", axis=1)
    uniques_df_top = pd.merge(
        left=uniques_df_top,
        right=uniques_df,
        left_on="Principal Component",
        right_on="Principal Component"
    )
    uniques_df_top = uniques_df_top[["Principal Component", "Score", "Rank"] + [cat for cat in uniques_df_top.columns if "_" in cat and not any(n in cat for n in ["4", "5"])]]
    uniques_df_top.to_csv("outputs/top_10_PCs.csv")

In [19]:
dfs = [
    "Ranked_PCs_" + str(n) + ".csv"
    for n in [1, 3, 4, 6, 7, 9]
]
is_shared_all(dfs)

   Principal Component   Model 1   Model 3   Model 4   Model 7   Model 9  \
24               PC-62         -         -         -  1.255665  0.051537   
11               PC-13         -  0.021294   0.63824  0.512523  0.033843   
8                PC-40  0.062688         -  0.381748  0.395343  0.019093   
16               PC-60         -         -  0.779287         -         -   
19               PC-14         -         -  0.213635  0.491027         -   
15               PC-11         -  0.015215  0.272449  0.295438         -   
17               PC-41         -         -   0.29782  0.242468         -   
13               PC-34         -  0.016383  0.183574  0.248079  0.029052   
25               PC-17         -         -         -  0.352279         -   
26               PC-10         -         -         -  0.277321         -   
18               PC-39         -         -   0.27221         -         -   
0                PC-55  0.184441  0.021802         -         -         -   
1           